### Shared Word

In [20]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize

In [21]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [22]:
lang_folder = "Turkish"  # Primary language
lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian

In [ ]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [23]:
lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
lang_pair_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish_And_Arabic (ara)_Etymologeek_Result_All.xlsx']

In [24]:
df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair

,search_word,dict_entry_main,language_main,definition_main,dict_entry,language,definition
0,mı,mıknatıs,Turkish,(Noun) Magnet.Magnet.,مِغْنَاطِيس,Arabic (ara),NaN
1,mı,mısdak,Turkish,(Noun) (obsolete) criterion.(obsolete) criterion.,مِصْدَاق,Arabic (ara),NaN
2,şey,şey,Turkish,(Noun Interjection) (slang) genital organ. Thi...,شَيْئاً,Arabic (ara),NaN
3,şey,şeytan,Turkish,(Noun) Devil.Devil.,شَيَّطَ,Arabic (ara),NaN
4,şey,şeytanî,Turkish,(Adjective) Devilish.Devilish.,شَيْطَانِيّ,Arabic (ara),NaN
...,...,...,...,...,...,...,...
362,avra,avrat,Turkish,(Noun) Wife.Wife.,عَوِرَ,Arabic (ara),NaN
363,molla,molla,Turkish,NaN,مَوْلًى,Arabic (ara),NaN
364,ruba,rubai,Turkish,"(Noun) Rubai; quatrain in classical Arabic, Pe...",رباع,Arabic (ara),"Four at a time, in a group of four, in fours."
365,كتاب,كتاب,Turkish,NaN,كتاب,Arabic (ara),"(obsolete). Book. Letter, note, paper, piece o..."


In [25]:
word_list = df_pair["dict_entry_main"].values.tolist()

In [32]:
# Select N Gram Type
sent_ngram_file = "Two" # Two, Three, Four, Five, Sentence

data_kind = f"{sent_ngram_file}gram" # Twogram, Threegram, Fourgram, Fivegram
if sent_ngram_file.lower() != "sentence":
    data_kind = f"{sent_ngram_file}gram"
    folder_path = "N Gram"
else:
    data_kind = "sentence"
    folder_path = "Sentence"

In [36]:
df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
df = df.head(100000)
df

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
99995,şeyin olumlu,125
99996,burada içki,125
99997,et vur,125
99998,peki dün,125


In [37]:
d_list  = df.iloc[:,0].values.tolist()

In [38]:
resultlist = []

In [39]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
resultlist = manager.list()

def word_in_wordgroup(d_list):
    mergelist = []
    try:
        word = d_list.split()
    except:
        pass
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist.append(d_list)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list) # string_word liste

In [48]:
result_list = list(resultlist)
df_result = pd.DataFrame(result_list)
df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
df_merge_result.drop_duplicates(inplace=True)
df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
df_merge_result.reset_index(drop=True, inplace=True)
df_merge_result

,twogram,frequency
0,hayır lütfen,11636
1,aynı şey,11019
2,kadar basit,10150
3,kadar emin,5919
4,selam millet,5475
...,...,...
212,yani ciddi,128
213,kafa vücut,128
214,tamam şey,128
215,tamam fakat,128


In [ ]:
df_merge_result.to_excel(f"{data_kind.capitalize()}")